In [1]:
# Colab Setup
!pip install openai diffusers transformers accelerate gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.7 MB/s eta 0:00:00


In [2]:
# Imports
import os
import torch
import gradio as gr
from diffusers import StableDiffusionPipeline
from openai import OpenAI
from google.colab import userdata  # Only for Colab

# API Key
api_key = userdata.get('Grok')  # Use your key stored in Colab
os.environ['OPENAI_API_KEY'] = api_key
client = OpenAI(base_url="https://api.groq.com/openai/v1")

#  Chatbot System Prompt
chat_model = "llama3-70b-8192"
system = """You are a creative and imaginative storyteller bot named Elara.
You write short, vivid, and engaging stories based on short prompts and moods.
Each story should be 4–6 lines long and match the selected genre/mood."""

# Load Stable Diffusion
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda" if torch.cuda.is_available() else "cpu")

#  Main Function: Story + Image
def generate_story_and_image(prompt, mood, guidance_scale=10.0, steps=50):
    # Step 1: Generate story
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Write a short {mood.lower()} story about: {prompt}"}
    ]
    story_response = client.chat.completions.create(
        model=chat_model,
        messages=messages
    )
    story = story_response.choices[0].message.content

    # Step 2: Generate image based on original prompt
    image = pipe(prompt, guidance_scale=guidance_scale, num_inference_steps=steps).images[0]

    return story, image

# 🎛 Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 📖 Elara: Story + Image Generator with Mood")

    with gr.Row():
        user_prompt = gr.Textbox(label="Enter a Creative Prompt", placeholder="e.g., a time-traveling fox in Paris")

    with gr.Row():
        mood = gr.Dropdown(
            choices=["Fantasy", "Funny", "Sad", "Sci-Fi", "Adventure", "Romantic"],
            value="Fantasy",
            label="Choose Story Mood"
        )

    with gr.Row():
        guidance = gr.Slider(7.5, 20.0, value=10.0, step=0.5, label="Image Guidance Scale")
        steps = gr.Slider(10, 100, value=50, step=5, label="Inference Steps")

    with gr.Row():
        story_output = gr.Textbox(label="📝 Elara's Story", lines=6)
        image_output = gr.Image(label="🎨 Illustration")

    submit = gr.Button("Generate Story + Image")
    submit.click(
        fn=generate_story_and_image,
        inputs=[user_prompt, mood, guidance, steps],
        outputs=[story_output, image_output]
    )

app.launch()

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://08068133a72fe80cea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
